In [ ]:
#DOTA数据集默认png格式图片，暂时把/root/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/mmrotate/datasets/dota.py里的png全改成bmp

from mmdet.apis import set_random_seed

from mmcv import Config
cfg = Config.fromfile('../../configs/oriented_rcnn/oriented_rcnn_r50_fpn_1x_dota_le90.py')

# Modify dataset type and path


cfg.load_from = '../../checkpoints/oriented_rcnn_r50_fpn_1x_dota_le90-6d2b2ce0.pth'

cfg.data.samples_per_gpu=1
cfg.data.workers_per_gpu=1

#classes = ('crab',)     #只有一中目标时，要这么写
#cfg.data.val.classes = classes
#cfg.model.roi_head.bbox_head[0].num_classes=1
#cfg.model.roi_head.bbox_head[1].num_classes=1

cfg.data_root = '/opt/data/private/shen/data/crab/'


cfg.data.test.ann_file = '/opt/data/private/shen/data/crab/test/txt_label_v2/'    #没有测试集标注文件，就和图片路径一样?
cfg.data.test.img_prefix = '/opt/data/private/shen/data/crab/test/images/'
#cfg.data.test.classes = classes

cfg.data.train.ann_file = '/opt/data/private/shen/data/crab/train/txt_label_v2/'   #train和val用的同一个数据集？
cfg.data.train.img_prefix = '/opt/data/private/shen/data/crab/train/images/'
#cfg.data.train.classes = classes

cfg.data.val.ann_file = '/opt/data/private/shen/data/crab/test/txt_label_v2/'
cfg.data.val.img_prefix = '/opt/data/private/shen/data/crab/test/images/'


'''
cfg.data_root = '/opt/data/private/shen/data/DOTA'

cfg.data.test.ann_file = '/opt/data/private/shen/data/DOTA/val/images/'   
cfg.data.test.img_prefix = '/opt/data/private/shen/data/DOTA/val/images/'

cfg.data.train.ann_file = '/opt/data/private/shen/data/DOTA/train/labelTxt-v1.0/'   
cfg.data.train.img_prefix = '/opt/data/private/shen/data/DOTA/train/images/'

cfg.data.val.ann_file = '/opt/data/private/shen/data/DOTA/val/labelTxt-v1.0/'
cfg.data.val.img_prefix = '/opt/data/private/shen/data/DOTA/val/images/'
'''



'''
cfg.data_root = '/opt/data/private/shen/data/split_ms_dota'

cfg.data.test.ann_file = '/opt/data/private/shen/data/split_ms_dota/test/annfiles/'   
cfg.data.test.img_prefix = '/opt/data/private/shen/data/split_ms_dota/test/images/'

cfg.data.train.ann_file = '/opt/data/private/shen/data/split_ms_dota/trainval/annfiles/'   
cfg.data.train.img_prefix = '/opt/data/private/shen/data/split_ms_dota/trainval/images/'

cfg.data.val.ann_file = '/opt/data/private/shen/data/split_ms_dota/trainval/annfiles/'
cfg.data.val.img_prefix = '/opt/data/private/shen/data/split_ms_dota/trainval/images/'
'''
cfg.work_dir = '/opt/data/private/shen/mmrotate-0.3.3/work_dir/crab'

cfg.optimizer.lr = 0.005
cfg.lr_config.warmup = None
cfg.runner.max_epochs = 1
cfg.log_config.interval = 2   #每下降10次展示一次loss

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1  #每迭代1次评估一次mAP
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1  #每迭代3次存储一次训练模型

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can also use tensorboard to log the training process
#cfg.log_config.hooks = [
#    dict(type='TextLoggerHook'),
#    dict(type='TensorboardLoggerHook')]

# We can initialize the logger for training and have a look
# at the final config used for training
cfg.model.roi_head.bbox_head.loss_bbox=dict(type='MyLoss', loss_weight=1.0)
print(f'Config:\n{cfg.pretty_text}')
cfg.dump(F'{cfg.work_dir}/crab.py')  #保存配置文件

In [2]:

import mmcv
import copy
import os.path as osp
import numpy as np

from mmrotate.datasets.builder import DATASETS
from mmrotate.datasets import build_dataset
from mmrotate.models import build_detector
from mmrotate.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

#大数据集载入要挺久

/opt/data/private/shen/mmdetection-2.26.0/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '
2023-01-15 12:03:56,673 - mmrotate - INFO - load checkpoint from local path: ../../checkpoints/oriented_rcnn_r50_fpn_1x_dota_le90-6d2b2ce0.pth
2023-01-15 12:03:57,091 - mmrotate - INFO - Start running, host: root@interactive58380, work_dir: /opt/data/private/shen/mmrotate-0.3.3/work_dir/crab
2023-01-15 12:03:57,093 - mmrotate - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook       

torch.Size([1, 3, 928, 1024])
tensor([[ 32.0326, 236.3941,  51.6702, 274.7767,  36.7318, 282.4196,  17.0942,
         244.0370],
        [ 32.0326, 236.3941,  51.6702, 274.7767,  36.7318, 282.4196,  17.0942,
         244.0370],
        [ 32.0326, 236.3941,  51.6702, 274.7767,  36.7318, 282.4196,  17.0942,
         244.0370],
        [ 32.0326, 236.3941,  51.6702, 274.7767,  36.7318, 282.4196,  17.0942,
         244.0370],
        [ 32.0326, 236.3941,  51.6702, 274.7767,  36.7318, 282.4196,  17.0942,
         244.0370],
        [ 32.0326, 236.3941,  51.6702, 274.7767,  36.7318, 282.4196,  17.0942,
         244.0370],
        [ 32.0326, 236.3941,  51.6702, 274.7767,  36.7318, 282.4196,  17.0942,
         244.0370],
        [ 32.0326, 236.3941,  51.6702, 274.7767,  36.7318, 282.4196,  17.0942,
         244.0370],
        [ 32.0326, 236.3941,  51.6702, 274.7767,  36.7318, 282.4196,  17.0942,
         244.0370],
        [ 32.0326, 236.3941,  51.6702, 274.7767,  36.7318, 282.4196,  17.0942,


2023-01-15 12:04:05,063 - mmrotate - INFO - Saving checkpoint at 1 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1/1, 1.3 task/s, elapsed: 1s, ETA:     0s

2023-01-15 12:05:17,003 - mmrotate - INFO - 
+--------------------+-----+------+--------+-------+
| class              | gts | dets | recall | ap    |
+--------------------+-----+------+--------+-------+
| plane              | 0   | 0    | 0.000  | 0.000 |
| baseball-diamond   | 0   | 0    | 0.000  | 0.000 |
| bridge             | 0   | 0    | 0.000  | 0.000 |
| ground-track-field | 0   | 0    | 0.000  | 0.000 |
| small-vehicle      | 20  | 20   | 0.800  | 0.797 |
| large-vehicle      | 87  | 72   | 0.828  | 0.818 |
| ship               | 0   | 1    | 0.000  | 0.000 |
| tennis-court       | 0   | 0    | 0.000  | 0.000 |
| basketball-court   | 0   | 0    | 0.000  | 0.000 |
| storage-tank       | 0   | 0    | 0.000  | 0.000 |
| soccer-ball-field  | 0   | 0    | 0.000  | 0.000 |
| roundabout         | 0   | 0    | 0.000  | 0.000 |
| harbor             | 0   | 0    | 0.000  | 0.000 |
| swimming-pool      | 0   | 0    | 0.000  | 0.000 |
| helicopter         | 0   | 0    | 0.000  | 0.000 |
+